<a href="https://colab.research.google.com/github/ghluque/Dolar-vs-inflacion/blob/main/DOLAR%2CUVAS%2CUVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backtesting y simulación en finanzas

# Dólar vs inflación en Argentina

In [ ]:
!pip install --upgrade plotly >/dev/null 2>&1 ; #Instalo Plotly para trabajar

In [ ]:
PrepareForWeb = False
#PrepareForWeb= True U: descomentar esta línea antes de guardar en Github

# Librerias


```

```





> Instalamos las librerias a utilizar



In [ ]:
import pandas as pd #U: leer y agrupar datos
import numpy as np #Para trabajar con matrices y funciones matematicas
import plotly.graph_objects as go #U: graficar con plotly
import plotly.express as px #U: graficar con plotly
import plotly #SEE: https://medium.com/@282abhishek/using-plotly-with-nbconvert-in-google-colab-96834c4f2850
if PrepareForWeb:
  plotly.offline.init_notebook_mode(connected=False)
else:
  plotly.io.renderers.default = 'colab'

In [ ]:
fecha_100 = "2023-12-12"

# DF UVAS

In [ ]:
uva_df_tables= pd.read_html('http://data.podemosaprender.org/finanzas/UVA_BCRA_2023.html') #Sacamos los datos de el siguiente link (estan en html)
uva_df= uva_df_tables[0] #A: sabemos que hay una sola!
uva_df.rename(columns={'Valor':'UVA', 'Fecha': 'fecha'}, inplace=True) #Renombramos las columnas del dataframe
uva_df.fecha= pd.to_datetime(uva_df.fecha, format='%Y-%m-%d') #Renombramos el dataframe y usamos la libreria de tiempo, cambiando el formato de fecha
uva_df.head() #Mostramos las primeras 5 filas

,fecha,UVA
0,2023-01-01,185.32
1,2023-01-02,185.32
2,2023-01-03,186.18
3,2023-01-04,186.47
4,2023-01-05,186.76


In [ ]:
uva_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fecha   362 non-null    datetime64[ns]
 1   UVA     362 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.8 KB


In [ ]:
uva_inicial = uva_df[uva_df['fecha'] == fecha_100]['UVA'].tolist()[0]

*¿Por qué elegimos trabajar con UVAS? Sabemos que las UVAS son bonos pero se calculan casi de la misma manera que el CER, el cual refleja la inflación.*

**Para hacer una comparativa, sería conveniente pasar los valores a Indices**

In [ ]:
uva_df['indice']=uva_df['UVA'] / uva_inicial *100 #Divido toda la columna venta por el primer dia que quiero comparar
uva_df

,fecha,UVA,indice
0,2023-01-01,185.32,42.677844
1,2023-01-02,185.32,42.677844
2,2023-01-03,186.18,42.875895
3,2023-01-04,186.47,42.942680
4,2023-01-05,186.76,43.009465
...,...,...,...
357,2023-12-24,450.97,103.855100
358,2023-12-25,450.97,103.855100
359,2023-12-26,450.97,103.855100
360,2023-12-27,458.03,105.480966


*De esta forma podemos ver que el indice = dia inicial x 100/ los valores de la columna 'UVA'*



# DF Dólar

In [ ]:
usd_blue_df= pd.read_csv("https://data.podemosaprender.org/finanzas/USD_BLUE_2023.csv", sep="\t") #Sacamos los datos del link (estan en formato csv)
usd_blue_df.fecha= pd.to_datetime(usd_blue_df.fecha, format='%Y/%m/%d') #Usamos la libreria de tiempo, cambiando el formato de fecha
usd_blue_df.head() #Mostramos las primeras 5 filas

,fecha,compra,venta
0,2023-01-02,342.0,342.0
1,2023-01-03,350.0,350.0
2,2023-01-04,350.0,350.0
3,2023-01-05,349.0,349.0
4,2023-01-06,350.0,350.0


**Ahora pasamos a indice el precio de venta del dolar**



In [ ]:
usd_blue_inicial = usd_blue_df[usd_blue_df['fecha'] == fecha_100]['venta'].tolist()[0] #El primer dia que quiero comparar

usd_blue_inicial

1020.0

In [ ]:
usd_blue_df['indice']=usd_blue_df['venta'] / usd_blue_inicial *100 #Divido toda la columna venta por el primer dia que quiero comparar
usd_blue_df

,fecha,compra,venta,indice
0,2023-01-02,342.0,342.0,33.529412
1,2023-01-03,350.0,350.0,34.313725
2,2023-01-04,350.0,350.0,34.313725
3,2023-01-05,349.0,349.0,34.215686
4,2023-01-06,350.0,350.0,34.313725
...,...,...,...,...
263,2023-12-20,945.0,945.0,92.647059
264,2023-12-21,940.0,940.0,92.156863
265,2023-12-22,945.0,945.0,92.647059
266,2023-12-26,945.0,945.0,92.647059


# Merge para unir DF's

**Comparamos datos con Merge**

In [ ]:
uva_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   fecha   362 non-null    datetime64[ns]
 1   UVA     362 non-null    float64       
 2   indice  362 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 8.6 KB


In [ ]:
indices_df = pd.merge(uva_df, usd_blue_df, on="fecha")
indices_df.rename(columns={'indice_x':'Indice_UVA', 'indice_y':'Indice_USD'}, inplace=True)
indices_df

,fecha,UVA,Indice_UVA,compra,venta,Indice_USD
0,2023-01-02,185.32,42.677844,342.0,342.0,33.529412
1,2023-01-03,186.18,42.875895,350.0,350.0,34.313725
2,2023-01-04,186.47,42.942680,350.0,350.0,34.313725
3,2023-01-05,186.76,43.009465,349.0,349.0,34.215686
4,2023-01-06,187.04,43.073947,350.0,350.0,34.313725
...,...,...,...,...,...,...
263,2023-12-20,445.74,102.650669,945.0,945.0,92.647059
264,2023-12-21,447.48,103.051378,940.0,940.0,92.156863
265,2023-12-22,449.22,103.452088,945.0,945.0,92.647059
266,2023-12-26,450.97,103.855100,945.0,945.0,92.647059


#Graficar con Plotly

In [ ]:
fig = px.line(indices_df, x="fecha", y=["Indice_UVA","Indice_USD"], title='Dólar vs Inflación 15-11-2023 = 100') #Graficamos definiendo las columnas del df definiendo cual es el eje x;y
fig.show()